## NIR Tutorial 

In [ ]:
# Plot images in line 
%matplotlib inline
import matplotlib
import cv2
# Set the desired figure size to get printed out 
matplotlib.rcParams["figure.figsize"] = (8.0, 8.0)
# Import PlantCV 
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/nir/original_image.jpg"
        self.debug = "plot"
        self.writeimg= False
        self.result = "./nir_tutorial_results"
        self.outdir = "."
        
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image
img, path, filename = pcv.readimage(args.image)

In [ ]:
# Read in the background image 
img_bkgrd = cv2.imread("img/tutorial_images/nir/background_average.jpg", flags=0)
# Manually plot the background image out since not using a PlantCV function 
pcv.plot_image(img_bkgrd)

In [ ]:
# Subtract the background image from the image with the plant. 

# Inputs: 
#   gray_img1 - Grayscale image data from which gray_img2 will be subtracted
#   gray_img2 - Grayscale image data which will be subtracted from gray_img1
bkg_sub_img = pcv.image_subtract(img, img_bkgrd)

In [ ]:
# Threshold the image of interest using the two-sided cv2.inRange function (keep what is between 50-190) 
bkg_sub_thres_img = cv2.inRange(bkg_sub_img, 50, 190)

# Since we are using an OpenCV function, we need to make it print 
if args.debug == 'print': 
    pcv.print_image(bkg_sub_thres_img, 'bkgrd_sub_thres.png')
elif args.debug == 'plot':
    pcv.plot_image(bkg_sub_thres_img)

In [ ]:
# Laplace filtering (identify edges based on 2nd derivative)

# Inputs:
#   gray_img - Grayscale image data 
#   k - Aperture size used to calculate the second derivative filter, 
#       specifies the size of the kernel (must be an odd integer)
#   scale - Scaling factor applied (multiplied) to computed Laplacian values 
#           (scale = 1 is unscaled) 
lp_img = pcv.laplace_filter(img, 1, 1)

In [ ]:
# Lapacian image sharpening, this step will enhance the darkness of the edges detected
lp_shrp_img = pcv.image_subtract(img, lp_img)

In [ ]:
# Sobel filtering
# 1st derivative sobel filtering along horizontal axis, kernel = 1)

# Inputs: 
#   gray_img = Grayscale image data 
#   dx - Derivative of x to analyze 
#   dy - Derivative of y to analyze 
#   k - Aperture size used to calculate 2nd derivative, specifies the size of the kernel and must be an odd integer
# NOTE: Aperture size must be greater than the largest derivative (k > dx & k > dy) 
sbx_img = pcv.sobel_filter(img, 1, 0, 1)

In [ ]:
# 1st derivative sobel filtering along vertical axis, kernel = 1)

sby_img = pcv.sobel_filter(img, 0, 1, 1)

In [ ]:
# Combine the effects of both x and y filters through matrix addition
# This will capture edges identified within each plane and emphasize edges found in both images

# Inputs:
#   gray_img1 - Grayscale image data to be added to gray_img2
#   gray_img2 - Grayscale image data to be added to gray_img1
sb_img = pcv.image_add(sbx_img, sby_img)

In [ ]:
# Use a lowpass (blurring) filter to smooth sobel image

# Inputs:
#   gray_img - Grayscale image data 
#   ksize - Kernel size (integer or tuple), (ksize, ksize) box if integer input,
#           (n, m) box if tuple input 
mblur_img = pcv.median_blur(sb_img, 1)

In [ ]:
# Invert the image so our background is white 

# Inputs:
#   gray_img - Grayscale image data 
mblur_invert_img = pcv.invert(mblur_img)

In [ ]:
# Combine the smoothed sobel image with the laplacian sharpened image
# combines the best features of both methods as described in "Digital Image Processing" by Gonzalez and Woods pg. 169

edge_shrp_img = pcv.image_add(mblur_invert_img, lp_shrp_img)

In [ ]:
# Perform thresholding to generate a binary image

# Inputs: 
#   gray_img - Grayscale image data 
#   threshold - Threshold value (0-255)
#   max_value - Value to apply above the threshold (255 = white)
#   object_type - 'light' (default) or 'dark'. If the object is lighter than 
#                 the background then standard thresholding is done. If the 
#                 object is darker than the background then inverse thresholding. 
tr_es_img = pcv.threshold.binary(edge_shrp_img, 145, 255, 'dark')

In [ ]:
# Do erosion with a 3x3 kernel

# Inputs: 
#   gray_img - Grayscale (usually binary) image data 
#   kernel - An odd integer that is used to build a kernel x kernel matrix
#            using np.ones. Must be greater than 1 to have an effect. 
#   i - An integer for the number of iterations, i.e. the number of consecutive
#       filtering passes 
e1_img = pcv.erode(tr_es_img, 3, 1)

In [ ]:
# Bring the two object identification approaches together.
# Using a logical OR combine object identified by background subtraction and the object identified by derivative filter.

# Inputs: 
#   bin_img1 - Binary image data to be compared in bin_img2
#   bin_img2 - Binary image data to be compared in bin_img1
comb_img = pcv.logical_or(e1_img, bkg_sub_thres_img)

In [ ]:
# Get masked image, Essentially identify pixels corresponding to plant and keep those.

# Inputs: 
#   rgb_img - RGB image data 
#   mask - Binary mask image data 
#   mask_color - 'black' or 'white'
masked_erd = pcv.apply_mask(img, comb_img, 'black')

In [ ]:
# Need to remove the edges of the image, we did that by generating a set of rectangles to mask the edges
# img is (254 X 320)
# Mask for the bottom of the image

# Inputs:
#   img - RGB or grayscale image data 
#   p1 - Point at the top left corner of the rectangle (tuple)
#   p2 - Point at the bottom right corner of the rectangle (tuple) 
#   color 'black' (default), 'gray', or 'white'
masked1, box1_img, rect_contour1, hierarchy1 = pcv.rectangle_mask(img, (120,184), (215,252))

In [ ]:
# Mask for the left side of the image

masked2, box2_img, rect_contour2, hierarchy2 = pcv.rectangle_mask(img, (1,1), (85,252))

In [ ]:
# Mask for the right side of the image

masked3, box3_img, rect_contour3, hierarchy3 = pcv.rectangle_mask(img, (240,1), (318,252))

In [ ]:
# Mask the edges

masked4, box4_img, rect_contour4, hierarchy4 = pcv.rectangle_mask(img, (1,1), (318,252))

In [ ]:
# Combine boxes to filter the edges and car out of the photo

bx12_img = pcv.logical_or(box1_img, box2_img)

In [ ]:
bx123_img = pcv.logical_or(bx12_img, box3_img)

In [ ]:
bx1234_img = pcv.logical_or(bx123_img, box4_img)

In [ ]:
# Invert this mask and then apply it the masked image.

inv_bx1234_img = pcv.invert(bx1234_img)

In [ ]:
edge_masked_img = pcv.apply_mask(masked_erd, inv_bx1234_img, 'black')

In [ ]:
# Identify objects

# Inputs:
#   img - RGB or grayscale image data for plotting
#   mask - Binary mask used for detecting contours
id_objects,obj_hierarchy = pcv.find_objects(edge_masked_img, inv_bx1234_img)

In [ ]:
# Define ROI

# Inputs: 
#   x - The x-coordinate of the upper left corner of the rectangle 
#   y - The y-coordinate of the upper left corner of the rectangle 
#   h - The height of the rectangle 
#   w - The width of the rectangle 
#   img - RGB or grayscale image to plot the ROI on 
roi1, roi_hierarchy= pcv.roi.rectangle(x=100, y=100, h=200, w=200, img=edge_masked_img)

In [ ]:
# Decide which objects to keep

# Inputs:
#   img - RGB or grayscale image data to display kept objects on 
#   roi_type - 'cutto' or 'partial' => include objexts that are partially inside or overlapping with the ROI 
#   roi_contour - contour of ROI, output from pcv.roi.rectangle in this case
#   object_contour - contour of objects, output from pcv.roi.rectangle in this case 
#   obj_hierarchy - heirarch of objects, output from pcv.find_objects function 
roi_objects, hierarchy5, kept_mask, obj_area = pcv.roi_objects(edge_masked_img, 'partial', roi1, roi_hierarchy, id_objects, obj_hierarchy)

In [ ]:
# Change the image to have 3 channels 
rgb_img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

# Use the object_composition function to outline the plant 
# Inputs:
#   img - RGB or grayscale image data for plotting 
#   contours - Contour list 
#   hierarchy - Contour hierarchy array 
grp_object, img_mask = pcv.object_composition(rgb_img, roi_objects, hierarchy5)

Now we can perform the analysis of pixelwise signal value and object shape attributes.


In [ ]:
# Perform signal analysis

# Inputs: 
#   gray_img - 8 or 16-bit grayscale image data 
#   mask - Binary mask made from selected contours 
#   bins - Number of classes to divide the spectrum into 
#   hisplot - If True, plots the histogram of intensity values 
#   filename - Name for output images 
nir_header, nir_data, nir_img = pcv.analyze_nir_intensity(img, kept_mask, 256, args.outdir + '/' + filename)

In [ ]:
# Perform shape analysis

# Inputs:
#   img - RGB or grayscale image data 
#   obj- Single or grouped contour object
#   mask - Binary image mask to use as mask for moments analysis 
#   filename - False (default) or image name. If defined, then print image
shape_header, shape_data, shape_img = pcv.analyze_object(rgb_img, grp_object, img_mask, args.outdir + '/' + filename)

In [ ]:
# Write shape and nir data to results file
result=open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
for row in shape_img:
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.write('\t'.join(map(str,nir_header)))
result.write("\n")
result.write('\t'.join(map(str,nir_data)))
result.write("\n")
for row in nir_img:
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.close()